In [ ]:
import math
import argparse

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from numpy import random
from torch.nn.parameter import Parameter
from OpenAttMultiGL.utils.dataset import dataset
from OpenAttMultiGL.utils.process import * 
from OpenAttMultiGL.model.GATNE.utils import *
#from mGCN_Toolbox.model.GATNE.walk import *
from OpenAttMultiGL.model.X_GOAL.evaluate import *
def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--input', type=str, default='Amazon',
                        help='Input dataset path')
    
    parser.add_argument('--features', type=str, default=None,
                        help='Input node features')

    parser.add_argument('--walk-file', type=str, default=None,
                        help='Input random walks')

    parser.add_argument('--epoch', type=int, default=10,
                        help='Number of epoch. Default is 100.')
    parser.add_argument('--lr', type=float, default=0.001, help='learning rate')

    parser.add_argument('--batch-size', type=int, default=64,
                        help='Number of batch_size. Default is 64.')

    parser.add_argument('--eval-type', type=str, default='all',
                        help='The edge type(s) for evaluation.')
    
    parser.add_argument('--schema', type=str, default=None,
                        help='The metapath schema (e.g., U-I-U,I-U-I).')

    parser.add_argument('--dimensions', type=int, default=200,
                        help='Number of dimensions. Default is 200.')

    parser.add_argument('--edge-dim', type=int, default=10,
                        help='Number of edge embedding dimensions. Default is 10.')
    
    parser.add_argument('--att-dim', type=int, default=20,
                        help='Number of attention dimensions. Default is 20.')

    parser.add_argument('--walk-length', type=int, default=10,
                        help='Length of walk per source. Default is 10.')

    parser.add_argument('--num-walks', type=int, default=20,
                        help='Number of walks per source. Default is 20.')

    parser.add_argument('--window-size', type=int, default=5,
                        help='Context size for optimization. Default is 5.')
    
    parser.add_argument('--negative-samples', type=int, default=5,
                        help='Negative samples for optimization. Default is 5.')
    
    parser.add_argument('--neighbor-samples', type=int, default=10,
                        help='Neighbor samples for aggregation. Default is 10.')

    parser.add_argument('--patience', type=int, default=5,
                        help='Early stopping patience. Default is 5.')
    
    parser.add_argument('--num-workers', type=int, default=16,
                        help='Number of workers for generating random walks. Default is 16.')
    
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

    return parser.parse_args()

def get_batches(pairs, neighbors, batch_size):
    n_batches = (len(pairs) + (batch_size - 1)) // batch_size

    for idx in range(n_batches):
        x, y, t, neigh = [], [], [], []
        for i in range(batch_size):
            index = idx * batch_size + i
            if index >= len(pairs):
                break
            x.append(pairs[index][0])
            y.append(pairs[index][1])
            t.append(pairs[index][2])
            neigh.append(neighbors[pairs[index][0]])
        yield torch.tensor(x), torch.tensor(y), torch.tensor(t), torch.tensor(neigh)


class GATNEModel(nn.Module):
    def __init__(
        self, num_nodes, embedding_size, embedding_u_size, edge_type_count, dim_a, features
    ):
        super(GATNEModel, self).__init__()
        self.num_nodes = num_nodes
        self.embedding_size = embedding_size
        self.embedding_u_size = embedding_u_size
        self.edge_type_count = edge_type_count
        self.dim_a = dim_a

        self.features = None
        if features is not None:
            self.features = features
            feature_dim = self.features.shape[-1]
            self.embed_trans = Parameter(torch.FloatTensor(feature_dim, embedding_size))
            self.u_embed_trans = Parameter(torch.FloatTensor(edge_type_count, feature_dim, embedding_u_size))
        else:
            self.node_embeddings = Parameter(torch.FloatTensor(num_nodes, embedding_size))
            self.node_type_embeddings = Parameter(
                torch.FloatTensor(num_nodes, edge_type_count, embedding_u_size)
            )
        self.trans_weights = Parameter(
            torch.FloatTensor(edge_type_count, embedding_u_size, embedding_size)
        )
        self.trans_weights_s1 = Parameter(
            torch.FloatTensor(edge_type_count, embedding_u_size, dim_a)
        )
        self.trans_weights_s2 = Parameter(torch.FloatTensor(edge_type_count, dim_a, 1))

        self.reset_parameters()

    def reset_parameters(self):
        if self.features is not None:
            self.embed_trans.data.normal_(std=1.0 / math.sqrt(self.embedding_size))
            self.u_embed_trans.data.normal_(std=1.0 / math.sqrt(self.embedding_size))
        else:
            self.node_embeddings.data.uniform_(-1.0, 1.0)
            self.node_type_embeddings.data.uniform_(-1.0, 1.0)
        self.trans_weights.data.normal_(std=1.0 / math.sqrt(self.embedding_size))
        self.trans_weights_s1.data.normal_(std=1.0 / math.sqrt(self.embedding_size))
        self.trans_weights_s2.data.normal_(std=1.0 / math.sqrt(self.embedding_size))

    def forward(self, train_inputs, train_types, node_neigh):
        if self.features is None:
            node_embed = self.node_embeddings[train_inputs]
            node_embed_neighbors = self.node_type_embeddings[node_neigh]
        else:
            node_embed = torch.mm(self.features[train_inputs], self.embed_trans)
            node_embed_neighbors = torch.einsum('bijk,akm->bijam', self.features[node_neigh], self.u_embed_trans)
        node_embed_tmp = torch.diagonal(node_embed_neighbors, dim1=1, dim2=3).permute(0, 3, 1, 2)
        node_type_embed = torch.sum(node_embed_tmp, dim=2)

        trans_w = self.trans_weights[train_types]
        trans_w_s1 = self.trans_weights_s1[train_types]
        trans_w_s2 = self.trans_weights_s2[train_types]

        attention = F.softmax(
            torch.matmul(
                torch.tanh(torch.matmul(node_type_embed, trans_w_s1)), trans_w_s2
            ).squeeze(2),
            dim=1,
        ).unsqueeze(1)
        node_type_embed = torch.matmul(attention, node_type_embed)
        node_embed = node_embed + torch.matmul(node_type_embed, trans_w).squeeze(1)

        last_node_embed = F.normalize(node_embed, dim=1)

        return last_node_embed


class NSLoss(nn.Module):
    def __init__(self, num_nodes, num_sampled, embedding_size):
        super(NSLoss, self).__init__()
        self.num_nodes = num_nodes
        self.num_sampled = num_sampled
        self.embedding_size = embedding_size
        self.weights = Parameter(torch.FloatTensor(num_nodes, embedding_size))
        self.sample_weights = F.normalize(
            torch.Tensor(
                [
                    (math.log(k + 2) - math.log(k + 1)) / math.log(num_nodes + 1)
                    for k in range(num_nodes)
                ]
            ),
            dim=0,
        )

        self.reset_parameters()

    def reset_parameters(self):
        self.weights.data.normal_(std=1.0 / math.sqrt(self.embedding_size))

    def forward(self, input, embs, label):
        n = input.shape[0]
        log_target = torch.log(
            torch.sigmoid(torch.sum(torch.mul(embs, self.weights[label]), 1))
        )
        negs = torch.multinomial(
            self.sample_weights, self.num_sampled * n, replacement=True
        ).view(n, self.num_sampled)
        noise = torch.neg(self.weights[negs])
        sum_log_sampled = torch.sum(
            torch.log(torch.sigmoid(torch.bmm(noise, embs.unsqueeze(2)))), 1
        ).squeeze()

        loss = log_target + sum_log_sampled
        return -loss.sum() / n


def train_model(network_data, feature_dic):
    vocab, index2word, train_pairs = generate(network_data, args.num_walks, args.walk_length, args.schema, file_name, args.window_size, args.num_workers, args.walk_file)

    edge_types = list(network_data.keys())

    num_nodes = len(index2word)
    edge_type_count = len(edge_types)
    epochs = args.epoch
    batch_size = args.batch_size
    embedding_size = args.dimensions
    embedding_u_size = args.edge_dim
    u_num = edge_type_count
    num_sampled = args.negative_samples
    dim_a = args.att_dim
    att_head = 1
    neighbor_samples = args.neighbor_samples

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    neighbors = generate_neighbors(network_data, vocab, num_nodes, edge_types, neighbor_samples)

    features = None
    if feature_dic is not None:
        feature_dim = len(list(feature_dic.values())[0])
        print('feature dimension: ' + str(feature_dim))
        features = np.zeros((num_nodes, feature_dim), dtype=np.float32)
        for key, value in feature_dic.items():
            if key in vocab:
                features[vocab[key].index, :] = np.array(value)
        features = torch.FloatTensor(features).to(device)

    model = GATNEModel(
        num_nodes, embedding_size, embedding_u_size, edge_type_count, dim_a, features
    )
    nsloss = NSLoss(num_nodes, num_sampled, embedding_size)

    model.to(device)
    nsloss.to(device)

    optimizer = torch.optim.Adam(
        [{"params": model.parameters()}, {"params": nsloss.parameters()}], lr=1e-4
    )
    
    best_score = 0
    test_score = (0.0, 0.0, 0.0)
    patience = 0
    for epoch in range(epochs):
        random.shuffle(train_pairs)
        batches = get_batches(train_pairs, neighbors, batch_size)
        print(batches.shape)
       #batches = get_batches(train_pairs, neighbors, batch_size)
        
        embes = model.node_embeddings#[t.test_id]
        #print(embes.shape)
        #print(t.dataset)
        data_iter = tqdm(
            batches,
            desc="epoch %d" % (epoch),
            bar_format="{l_bar}{r_bar}",
        )
        batch = torch.tensor(args.batch_size)
        avg_loss = 0.0
        l = []
        l.append(args.batch_size)
        l.append(2)
        l.append(args.neighbor_samples)
        #args.neighbor_samples
        node_neigh = torch.tensor(l)
        print(l)
        for i, data in enumerate(data_iter):
            optimizer.zero_grad()
            #loss = criterion(model[t.train_id], t.labels[t.train_id])
            embs = model(data[0].to(device), data[2].to(device), data[3].to(device),)
            #print(data[0].to(device).shape)
            #print(data[2].to(device).shape)
            #print(data[3].to(device).shape)
            
            #print(data[1].to(device).shape)
            loss = nsloss(data[0].to(device), embs, data[1].to(device))
            #loss = nsloss(batch, embs, batch)
            #print(loss.shape)
            loss.backward()
            optimizer.step()

           # avg_loss += loss.item()

            #if i % 5000 == 0:
               # post_fix = {
              #      "epoch": epoch,
              #      "iter": i,
              #      "avg_loss": avg_loss / (i + 1),
              #      "loss": loss.item(),
              #  }
              #  data_iter.write(str(post_fix))

        #final_model = dict(zip(edge_types, [dict() for _ in range(edge_type_count)]))
        #for i in range(num_nodes):
            #train_inputs = torch.tensor([i for _ in range(edge_type_count)]).to(device)
            #train_types = torch.tensor(list(range(edge_type_count))).to(device)
            #node_neigh = torch.tensor(
                #[neighbors[i] for _ in range(edge_type_count)]
            #).to(device)
            #node_emb = model(train_inputs, train_types, node_neigh)
            #for j in range(edge_type_count):
                #final_model[edge_types[j]][index2word[i]] = (
                    #node_emb[j].cpu().detach().numpy()
                #)

        #valid_aucs, valid_f1s, valid_prs = [], [], []
        #test_aucs, test_f1s, test_prs = [], [], []
        for i in range(edge_type_count):
            if args.eval_type == "all" or edge_types[i] in args.eval_type.split(","):
                macro,micro = evaluate_GATNE(
                    final_model[edge_types[i]],
                    t.testing_true_data_by_edge[edge_types[i]],
                    t.testing_false_data_by_edge[edge_types[i]],
                    t.num_classes
                )
                #test_aucs.append(tmp_auc)
                #test_f1s.append(tmp_f1)
                #test_prs.append(tmp_pr)
                
                
        test_embs = model.node_embeddings#[t.test_id]  
        test_lbls = torch.argmax(t.labels[t.test_id], dim=1)
        nmi = run_kmeans(test_embs, test_lbls, t.num_classes)
        sim = run_similarity_search(test_embs, test_lbls)
        print("Epoch: ", epoch)
        print("Macro_F1:", macro)
        print("Micro_F1:", micro)
        print("NMI:", nmi)
        print("SIM:", sim)

        #average_auc = np.mean(test_aucs)
        #average_f1 = np.mean(test_f1s)
        #average_pr = np.mean(test_prs)

        #cur_score = np.mean(valid_aucs)
        #if cur_score > best_score:
           # best_score = cur_score
          #  test_score = (average_auc, average_f1, average_pr)
           # patience = 0
        #else:
            #patience += 1
            #if patience > args.patience:
                #print("Early Stopping")
                #break
    #pred = model.max(1).indices
    #macro,micro,nmi,sim = evaluate(embs,t.train_id,t.valid_id,t.test_id,t.gcn_labels)
    #nmi = run_kmeans(t.labels,pred, nb_classes)
    #sim = run_similarity_search(t.labels,pred)
    return macro,micro,nmi,sim


if __name__ == "__main__":
    args = parse_args()
    file_name = args.input
    print(args.lr)
    if args.features is not None:
        feature_dic = load_txt_feature_data(args.features)
    else:
        feature_dic = None
    t = dataset(file_name)
    #training_data_by_type = load_training_data("mGCN_Toolbox/data/GATNE/"+file_name + "/train.txt")
    #valid_true_data_by_edge, valid_false_data_by_edge = load_valid_data(
        #"mGCN_Toolbox/data/GATNE/" +file_name + "/valid.txt"
    #)
    #testing_true_data_by_edge, testing_false_data_by_edge = load_testing_data(
        #"mGCN_Toolbox/data/GATNE/"+ file_name + "/test.txt"
    #)
    #print(t.dataset[0])

    #macro,micro,nmi,sim = train_model(t.training_data_by_type, feature_dic)
    macro,micro,nmi,sim = train_model(t.training_data_by_type, feature_dic)

    #print("Macro_F1:", macro)
    #print("Micro_F1:", micro)
    #print("NMI: ", nmi)
    #print("SIM: ", sim)



In [2]:
from torch.nn.parameter import Parameter
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from numpy import random

node_embeddings = Parameter(torch.FloatTensor(50, 200))

In [3]:
node_embeddings.shape

torch.Size([50, 200])

In [6]:
import tqdm
data_iter = tqdm(
            500,
            desc="epoch %d" % (500),
            bar_format="{l_bar}{r_bar}",
            )

TypeError: 'module' object is not callable